# Road name analysis
*April 22, 2022*

## Pitch

With new (or sometimes old) information coming to light about key figures in the founding of Canada, how we name our roads, landmarks, and institutions is a matter of public interest. This is more timely than ever: Egerton Ryerson, an architect of residential schools in Canada, recently had his name removed from a highly recognizably university.

Using StatsCan's road network database, we can look at:
1. How many roads in Canada are named for architects of residential schools.
2. How many of those roads have been changed since 2011.

## Methodology

How many street names have been changed from 2016 to 2021? There is a new road network file released with the census, and this is a topic on many peoples' minds. We can approach this in a few ways, but they all start by bringing in 2011 and 2021 data, and comparing the names to see which ones have changed. Let's start by importing geopandas, pandas, and the regex module.

In [1]:
import geopandas
import pandas as pd
import re

### Reading in and preparing the data

Now we'll read in these two (massive) datasets straight from StatsCan (yes, they take a long time to read in, at least 10 minutes for both).

In [2]:
raw_2011 = geopandas.read_file("https://www12.statcan.gc.ca/census-recensement/2011/geo/RNF-FRR/files-fichiers/grnf000r11a_e.zip")

In [3]:
raw_2021 = geopandas.read_file("https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/RNF-FRR/files-fichiers/lrnf000r21a_e.zip")

Let's take a quick look at how big these datasets are, so we have an idea.

In [4]:
display(raw_2011.shape)
display(raw_2021.shape)

(1973932, 28)

(2242117, 26)

There are a couple million records in each year. Now, before we start comparing them over time, we need to be sure there's an ID column that will let us do that. Let's check to see if we can compare names through time. We'll sort by NGD_ID, which looks to be a column with a unique ID for each road. We'll take a look at the column names to see what might work.

In [5]:
raw_2011.head(3)
raw_2021.head(3)

,OBJECTID,NGD_UID,NAME,TYPE,DIR,AFL_VAL,ATL_VAL,AFR_VAL,ATR_VAL,CSDDGUID_L,...,CSDTYPE_R,PRDGUID_L,PRUID_L,PRNAME_L,PRDGUID_R,PRUID_R,PRNAME_R,RANK,CLASS,geometry
0,1,5792582,des 60,RANG,None,195,195,182,194,2021A00052457050,...,MÉ,2021A000224,24,Quebec / Québec,2021A000224,24,Quebec / Québec,4,23,"LINESTRING (7650137.494 1271488.229, 7650168.1..."
1,2,4744971,Township Road 734,None,None,None,None,None,None,2021A00054819006,...,MD,2021A000248,48,Alberta,2021A000248,48,Alberta,5,22,"LINESTRING (4530157.094 2497211.194, 4530155.7..."
2,3,1935694,733 Grid,RD,None,None,None,None,None,2021A00054706063,...,RM,2021A000247,47,Saskatchewan,2021A000247,47,Saskatchewan,4,23,"LINESTRING (5262271.166 1735451.494, 5263851.4..."


As it turns out, the NGD_UID column is persistent through time, so we should be able to compare on that. Let's take a look.

In [6]:
display(raw_2011[["NGD_UID","NAME"]].sort_values("NGD_UID").head(5))
display(raw_2021[["NGD_UID","NAME"]].sort_values("NGD_UID").head(5))

,NGD_UID,NAME
1948572,1,Nazarene
1343212,100,Mabel
1648561,1000,Bedard
598762,100000,Connaught
355794,100001,1st


,NGD_UID,NAME
1686497,1,Nazarene
312421,100,Mabel
2111576,1000,Bedard
731670,100000,Connaught
1622872,100001,1st


Sure enough, it looks like we can use this column to compare changes through time. Before we get into it, let's capitalize all the street names so we don't get into trouble with cases, and add a "YEAR" column to each dataset so we can concatenate these two datasets together.

In [7]:
raw_2011["YEAR"] = "2011"
raw_2021["YEAR"] = "2021"

Now we'll concat the data together, and pivot to show the names from 2016 and 2021 side by side.

In [8]:
combined = pd.concat([raw_2011, raw_2021])

pivot = (combined
         .pivot(columns="YEAR", index="NGD_UID", values="NAME")
         .dropna()
         )

pivot.head(3)

YEAR,2011,2021
NGD_UID,,
1,Nazarene,Nazarene
100,Mabel,Mabel
1000,Bedard,Bedard


Now, we'll clean the street names a bit. We want to replace accented characters with non-accented ones, since it appears that often between the two years, the only differences are in accented characters. We'll also replace MOUNT with MT. and likewise for SAINT.

In [9]:
for year in ["2011", "2021"]:
    pivot[year] = (pivot[year]
                   .replace({
                        "\.|,|'|-": "",
                        "É|È|Ê": "E",
                        "Ô": "O",
                        "MOUNT ": "MT ",
                        "SAINT ": "ST ",
                        "À|Â": "A",
                        "Û": "U",
                        " ": ""
                    }, regex=True)
                   .str.upper()
    )
    
pivot.head()

YEAR,2011,2021
NGD_UID,,
1,NAZARENE,NAZARENE
100,MABEL,MABEL
1000,BEDARD,BEDARD
100000,CONNAUGHT,CONNAUGHT
100001,1ST,1ST


Now we're going to add a new column that tells us if the 2011 name matches the 2021 name or not. We'll store this as a boolean.

In [10]:
pivot["CHANGED?"] = pivot.apply(lambda x: not bool(re.search(x["2011"], x["2021"])), axis=1)

pivot.head()

YEAR,2011,2021,CHANGED?
NGD_UID,,,
1,NAZARENE,NAZARENE,False
100,MABEL,MABEL,False
1000,BEDARD,BEDARD,False
100000,CONNAUGHT,CONNAUGHT,False
100001,1ST,1ST,False


Now we define a list of names we want to search for. These names include some well-known architects of residential schools, but also some lesser known names that were found after a brief bit of research. Here are some of the lesser-known names found:
* [Frank Oliver](https://en.wikipedia.org/wiki/Frank_Oliver_(politician))
* [Vital-Justin Grandin](https://en.wikipedia.org/wiki/Vital-Justin_Grandin)
* [Nicholas Flood Davin](https://en.wikipedia.org/wiki/Nicholas_Flood_Davin)
* [Charles Bagot](https://en.wikipedia.org/wiki/Charles_Bagot)
* [James Girty](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [James Hayt](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [Joshua Mauger](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [Walter Patterson](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [William Jarvis](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [Joseph Brant](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [John McDonnell](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)
* [Peter Van Alstine](https://www.thecanadianencyclopedia.ca/en/article/black-enslavement)


In [11]:
names = [
    "DUNDAS",
    "RUSSELL",
    "MACDONALD",
    "RYERSON",
    "CORNWALLIS",
    "MCGILL",
    "EGERTON",
    "BOWELL",
    "MACKENZIE",
    "REED",
    "BAGOT",
    "DAVIN",
    "LANGEVIN",
    "GRANDIN",
    "OLIVER",
    "GIRTY",
    "HAYT",
    "MAUGER",
    "JARVIS",
    "BRANT",
    "VAN ALSTINE",
    "MCDONNELL",
    "PATTERSON"
    ]

We're going to join these together in a string so we can plug it into a regex search method to identify only those street names that we're interested in finding out if they've changed or not. We insert some \b tags to ensure we're not getting matches like MCGILLIVRAY when we want MCGILL (for example).

In [12]:
regex_string = r"\b|\b".join(names)
regex_string = r"\b" + regex_string + r"\b"

Now we actually perform the search, and show streets that match this criteria.

In [13]:
of_interest = pivot[pivot["2011"].str.contains(regex_string, regex=True)].sort_index()

For fun, let's see how many of these have changed since 2011, as this includes all streets with these names, not just those that have changed.

In [14]:
of_interest[of_interest["CHANGED?"] == True].head(5)

YEAR,2011,2021,CHANGED?
NGD_UID,,,
1209804,MACDONALD,MCDONALD,True
1412981,CORNWALLIS,LEGACY,True
1412988,CORNWALLIS,LEGACY,True
1412993,CORNWALLIS,LEGACY,True
1412995,CORNWALLIS,LEGACY,True


You'll notice here that there are some changes that aren't really changes, but more likely typos (MACDONALD was probably not changed to MCDONALD). Let's try to fix that.

In [15]:
disambig_list = ["MACKENZIE", "MCKENZIE", "MCDONALD", "MACDONALD", "RUSSEL", "RUSSELL"]

of_interest.loc[(of_interest["2011"].isin(disambig_list)) & (of_interest["2021"].isin(disambig_list)), "CHANGED?"] = False
               
of_interest[of_interest["CHANGED?"] == True]

YEAR,2011,2021,CHANGED?
NGD_UID,,,
1412981,CORNWALLIS,LEGACY,True
1412988,CORNWALLIS,LEGACY,True
1412993,CORNWALLIS,LEGACY,True
1412995,CORNWALLIS,LEGACY,True
1412997,CORNWALLIS,LEGACY,True
1413006,CORNWALLIS,LEGACY,True
1413017,CORNWALLIS,LEGACY,True
1413033,CORNWALLIS,LEGACY,True
1850772,BRANT,COLDWATER,True


You'll notice those rows are gone now!

Next, we're now going to use this `of_interest` list to filter our master dataset for roads of interest, changed or not. First, we create a list from our index, which contains the unique IDs for each of these road arcs.

In [16]:
id_list = of_interest[["2021"]].index

Now, we filter our raw data into a new dataframe called `filtered` that only shows roads of interest, but will show us more data, and the original, unaltered names of the roads (plus what kind of road they are, what province they're in etc.).

In [17]:
filtered = (raw_2021
            .loc[raw_2021["NGD_UID"].isin(id_list), ["NAME", "TYPE", "DIR", "PRNAME_L", "NGD_UID", "CSDUID_L", "CSDNAME_L", "geometry"]]
            .set_index("NGD_UID")
            .join(of_interest[["CHANGED?"]])
            .join(raw_2011.set_index("NGD_UID").loc[:, "NAME"], rsuffix="2011")
            .loc[:, ["NAME2011", "NAME", "DIR", "TYPE", "PRNAME_L", "CSDUID_L", "CSDNAME_L", "CHANGED?", "geometry"]]
            )

We'll also clean up our province names, which have both english and french in them, separated by a /.

In [18]:
filtered["PRNAME_L"] = filtered["PRNAME_L"].apply(lambda x: x.split(" / ")[0])
filtered["TYPE"] = filtered["TYPE"].fillna("").str.capitalize()

filtered

,NAME2011,NAME,DIR,TYPE,PRNAME_L,CSDUID_L,CSDNAME_L,CHANGED?,geometry
NGD_UID,,,,,,,,,
429664,MacDonald,MacDonald,None,St,Ontario,3518020,Scugog,False,"LINESTRING (7249740.871 986092.491, 7249637.22..."
391663,McGill,McGill,None,Rue,Quebec,2466023,Montréal,False,"LINESTRING (7632417.177 1244716.406, 7632371.5..."
858330,Macdonald,Macdonald,None,Ave,British Columbia,5915025,Burnaby,False,"LINESTRING (4025113.954 2002985.889, 4025091.6..."
707888,Dundas,Dundas,W,St,Ontario,3521005,Mississauga,False,"LINESTRING (7207341.883 916541.909, 7207312.35..."
391744,McGill,McGill,None,Rue,Quebec,2466023,Montréal,False,"LINESTRING (7632580.017 1244688.609, 7632514.6..."
...,...,...,...,...,...,...,...,...,...
4593788,Patterson,Patterson,None,Rd,British Columbia,5933037,Thompson-Nicola I (Blue Sky Country),False,"LINESTRING (4208924.426 2094135.983, 4208903.2..."
3175318,Oliver,Oliver,None,Terr,British Columbia,5919021,Ladysmith,False,"LINESTRING (3957306.794 2004219.871, 3957296.5..."
1982119,Bagot,Bagot,None,Rue,Quebec,2494068,Saguenay,False,"LINESTRING (7731052.551 1609830.471, 7730996.6..."


In [19]:
filtered.to_clipboard()

Now we have our dataset that we can manipulate in a bunch of different ways!

### Counts by census subdivision

To start, I want to visualize this data by census subdivision, showing what percentage of "problematic" street names have been changed and how many "problematic" street names there are. I'll do this using `pd.pivot_table` and the count aggfunc.

In [20]:
counts = (filtered
          .pivot_table(index=["CSDUID_L", "CSDNAME_L"], columns="CHANGED?", values="NAME2011", aggfunc="count")
          .fillna(0)
          .reset_index()
          .set_index("CSDUID_L")
          )

counts.head()

CHANGED?,CSDNAME_L,False,True
CSDUID_L,,,
1001370,Carbonear,2.0,0.0
1001409,Bay Roberts,1.0,0.0
1001519,St. John's,13.0,0.0
1001542,Mount Pearl,1.0,0.0
1003019,St. Alban's,0.0,1.0


Now let's calculate the % of streets that have changed in each CSD from these raw counts and put it into a new column. We'll also sort on this new column.

In [21]:
counts["%_changed"] = round(counts[True] / counts.sum(axis=1) *100, 2)
counts = counts.sort_values("%_changed", ascending=False)

counts.head()

C:\Users\dexmc\AppData\Local\Temp\ipykernel_53892\3263339586.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  counts["%_changed"] = round(counts[True] / counts.sum(axis=1) *100, 2)


CHANGED?,CSDNAME_L,False,True,%_changed
CSDUID_L,,,,
6105016,Hay River,0.0,2.0,100.0
4802031,Newell County,0.0,1.0,100.0
4803004,Cardston,0.0,7.0,100.0
1003019,St. Alban's,0.0,1.0,100.0
4807019,Stettler County No. 6,0.0,3.0,100.0


### Adding a list of names of streets that have been changed

For our map, it'd be nice to have a string containing the original names of some roads that have been changed. Let's add that to our counts dataframe. First we get the list of names we're interested in that have been changed since 2011 and set the index to be CSDUID, which is a unique ID given to each census subdivision.

You'll notice the L appended to the CSDUID here. There's also a CSDUID_R column. In some cases, street arcs may go from one census subdivision to another, and therefore CSDUID_L and CSDUID_R may be different for each (ie. L tells us which CSDUID the road starts in, R where it ends in.). Here we use CSDUID_L consistently, which is really the best we can do here.

In [22]:
changed_list = filtered[filtered["CHANGED?"] == True].set_index("CSDUID_L")

It's nice to have the type of street (road, street, avenue) after each name, so we'll add a new column that concatenates those together.

In [23]:
changed_list["FULL_NAME"] = (changed_list["NAME2011"].astype(str) + " " + changed_list["TYPE"]).str.strip()

Now we group by CSDUID, and we concatenate all unique road names together into a string.

(`set()` is a very handy little function that converts a list to a set, which can only have unique values. It's an easy way to strip out duplicates from a list)

In [24]:
changed_list = changed_list[["FULL_NAME"]].groupby('CSDUID_L').transform(lambda x: ', '.join(set(x)))

changed_list.head()

,FULL_NAME
CSDUID_L,
1217030,Cornwallis St
3520005,"Russell St, Jarvis Way"
3520005,"Russell St, Jarvis Way"
3520005,"Russell St, Jarvis Way"
4807019,MacDonald


Now, let's join this changed_list dataframe to our counts dataframe and fill in NaN values. We'll also rename columns to make them a little more clear.

In [25]:
counts = counts.join(changed_list)
counts["FULL_NAME"] = counts["FULL_NAME"].fillna("")
counts.columns = ["CSD Name", "Changed street names", "Not changed", "Changed", "Percent changed"]

counts.head()

,CSD Name,Changed street names,Not changed,Changed,Percent changed
CSDUID_L,,,,,
1001370,Carbonear,2.0,0.0,0.0,
1001409,Bay Roberts,1.0,0.0,0.0,
1001519,St. John's,13.0,0.0,0.0,
1001542,Mount Pearl,1.0,0.0,0.0,
1003019,St. Alban's,0.0,1.0,100.0,Macdonald Rd


That's it for now.

\-30\-